In [48]:
import cv2
import numpy as np

def calculate_ndvi(image_path):
    # Read the image
    img = cv2.imread(r"C:\Users\Admin\Downloads\Major Project\rice_leaf_diseases\rice_leaf_diseases\Bacterial leaf blight\DSC_0703.JPG")
    # Simulate Near-Infrared (NIR) and Red channels
    nir = img[:, :, 2]  # Assuming NIR is in the Red channel
    red = img[:, :, 0]  # Assuming Red is in the Blue channel
    # Calculate NDVI (Normalized Difference Vegetative Index)
    ndvi = (nir.astype(float) - red.astype(float)) / (nir + red + 1e-6)
    return np.mean(ndvi)  # Return average NDVI

ndvi_value = calculate_ndvi("leaf_image.jpg")
print("NDVI Value:", ndvi_value)


NDVI Value: 33605.87237668407


In [56]:
#Calculating Index
import cv2
import numpy as np

def calculate_indices(image_path):
    # Read the image
    img = cv2.imread(image_path)
    
    # Convert to float for calculations
    img = img.astype(np.float32) / 255.0  

    # Extract R, G, B channels
    R = img[:, :, 2]
    G = img[:, :, 1]
    B = img[:, :, 0]

    '''
    Compute VARI(Visible Atmospherically Resistant Index), 
    GLI (Green Leaf Index),
    ExG (Excess Green)
    '''
    vari = (G - R) / (G + R - B + 1e-6)  # Avoid division by zero
    gli = (2 * G - R - B) / (2 * G + R + B + 1e-6)
    exg = 2 * G - R - B

    return np.mean(vari), np.mean(gli), np.mean(exg)

# Example usage
image_path = r"C:\Users\Admin\Downloads\Major Project\rice_leaf_diseases\rice_leaf_diseases\Bacterial leaf blight\DSC_0703.JPG"
vari_value, gli_value, exg_value = calculate_indices(image_path)

print(f"VARI: {vari_value:.2f}")
print(f"GLI: {gli_value:.2f}")
print(f"ExG: {exg_value:.2f}")


VARI: -0.04
GLI: 0.06
ExG: 0.08


In [39]:
import cv2
import numpy as np
import pandas as pd
import os

def calculate_indices(image_path):
    """
    Calculate VARI, GLI, and ExG indices from an RGB image.
    
    Args:
        image_path (str): Path to the image file.
    
    Returns:
        tuple: (VARI, GLI, ExG) mean values.
    """
    # Read the image
    img = cv2.imread(image_path)
    
    # Check if image is loaded properly
    if img is None:
        print(f"Error: Unable to read image at {image_path}")
        return None, None, None
    
    # Convert to float for calculations
    img = img.astype(np.float32) / 255.0  

    # Extract R, G, B channels
    R = img[:, :, 2]
    G = img[:, :, 1]
    B = img[:, :, 0]

    # Compute vegetation indices
    vari = (G - R) / (G + R - B + 1e-6)  # Avoid division by zero
    gli = (2 * G - R - B) / (2 * G + R + B + 1e-6)
    exg = 2 * G - R - B

    return np.mean(vari), np.mean(gli), np.mean(exg)

# 📌 Compute indices for all images in a dataset folder
def process_dataset(image_folder, output_csv):
    """
    Process all images in a folder and save computed indices to a CSV file.
    
    Args:
        image_folder (str): Path to the folder containing images.
        output_csv (str): Path to save the output CSV file.
    """
    data = []
    
    for file_name in os.listdir(image_folder):
        if file_name.endswith(('.jpg', '.png', '.jpeg')):  # Check for valid image files
            image_path = os.path.join(image_folder, file_name)
            vari, gli, exg = calculate_indices(image_path)
            
            if vari is not None:  # Ensure valid computation
                data.append([file_name, vari, gli, exg])
    
    # Convert to DataFrame and save
    df = pd.DataFrame(data, columns=["Image_Name", "VARI", "GLI", "ExG"])
    df.to_csv(output_csv, index=False, float_format="%.2f")
    print(f"Dataset saved to {output_csv}")

# Example Usage
image_folder = r"C:\Users\Admin\Downloads\Major Project\rice_leaf_diseases\rice_leaf_diseases\blast"  # Update with your image folder path
output_csv = "vegetation_indices.csv"
process_dataset(image_folder, output_csv)


Dataset saved to vegetation_indices.csv


In [52]:
# Load vegetation indices and soil test data
vegetation_data = pd.read_csv("vegetation_indices.csv")
soil_data = pd.read_csv("npk_data.csv")  # Replace with actual soil test CSV

# Merge data using image names
final_dataset = pd.merge(soil_data, vegetation_data, left_on="Image_Path", right_on="Image_Name", how="inner")

# Drop unnecessary columns
final_dataset.drop(columns=["Image_Name"], inplace=True)

# Save the final dataset
final_dataset.to_csv("final_npk_dataset.csv", index=False)

print("Final dataset saved as final_npk_dataset.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'npk_data.csv'